### Creating and Running Threads
To create a thread we can either extend the Thread class or pass Runnable to the Thread constructor.

```java
// Extend Thread class
public class SampleThread extends Thread {
	@Override
	public void run() {
		System.out.println("Thread is running");
	}

	public static void main(String[] args) {
		SampleThread thread = new SampleThread();
		thread.start();
	}
}
```

Or
```java
public static void main(String[] args) {
		Runnable r = new Runnable() {
			@Override
			public void run() {
				System.out.println("Thread is running");
			}
		};
		
		Thread t = new Thread(r);
		t.start();
	}
```

We can assign a name to the Thread by using the constructor which accepts name as the second parameter.
```java
Thread t = new Thread(r, "Thread A");
t.start();
```

In order to get the current thread in execution:
```java
public static void main(String[] args) {
		Thread t1 = new Thread(() -> {
			System.out.println("Running thread: " + Thread.currentThread().getName());
		}, "Thread 1");
		t1.start();

		Thread t2 = new Thread(() -> {
			System.out.println("Running thread: " + Thread.currentThread().getName());
		}, "Thread 2");
		t2.start();

		Thread t3 = new Thread(() -> {
			System.out.println("Running thread: " + Thread.currentThread().getName());
		}, "Thread 3");
		t3.start();
	}
```

### Race Condition and Atomicity
Many operations in Java, consist of smaller operations. Operation as small as addition `a +=1` is actually composed of the following instructions:
- Get data from memory
- Add one to data
- Put data back to the memory

When multiple threads do the same operations, the above listed steps may get interleaved giving incorrect result. For example:
```java
public class AtomicOperation {
	public static void main(String[] args) {
		SharedUnsafeObject obj = new SharedUnsafeObject();

		for (int i = 0; i < 5; i++) {
			new Thread(() -> {
				obj.incrementCounter();
			}).start();
		}
		
		try {
			Thread.sleep(1000);
			System.out.println(obj.getCounter()); // Output not guaranteed as 5
		} catch (InterruptedException e) {
			e.printStackTrace();
		}
	}
}

class SharedUnsafeObject {
	private long counter = 0;

	public void incrementCounter() {
		this.counter++;
	}

	public long getCounter() {
		return this.counter;
	}
}
```

One possible solution is to use Atomic classes which do operations in one step:
```java
class SharedSafeObject {
	private AtomicLong counter;
	
	public void incrementCounter() {
		this.counter.incrementAndGet();
	}
	
	public long getCounter() {
		return this.counter.get();
	}
}
```

Another solution is to use synchronized blocks. 

### Interrupts
An interrupt is a signal to a thread to indicate that it should stop what it is currently doing and do something else. It is upto the thread to act upon the interrrupt or completely ignore it.

```java
Thread t = new Thread(() -> {
    int i = 0;
    for (; i < 1000000; i++) {
        if (Thread.interrupted()) {
            break;
        }
    }

    System.out.println("Interrupted at " + i);
});

t.start();

Thread.sleep(2); // Let the thread t to run for some time

t.interrupt(); // Now interrupt the thread t
```

some Thread methods also respond to interruption like the `Thread.sleep` method. In that case `InterruptedException` is thrown

```java
Thread t = new Thread(() -> {
    try {
        Thread.sleep(5000);
    } catch (InterruptedException e) {
        System.out.println("Thread was interrupted while sleeping");
    }
});

t.start();

Thread.sleep(1000); // Let the thread t to run for some time

t.interrupt(); // Now interrupt the thread t
```

**Interruption Flag:** The interrupt mechanism is implemented using an internal flag known as the interrupt status. Invoking `Thread.interrupt` sets this flag. When a thread checks for an interrupt by invoking the static method `Thread.interrupted`, interrupt status is cleared. The non-static `isInterrupted` method, which is used by one thread to query the interrupt status of another, does not change the interrupt status flag.

### Join
Calling `t.join()` causes the current thread to pause execution till the threat t completes execution. Similar to sleep, join also responds to interruption. 

```java
Thread[] threads = new Thread[5];
for(int i=0; i<5; i++){
    threads[i] = new Thread(new Runnable(){
        void run(){
            try {
                Thread.sleep(5000);
            } catch (InterruptedException e) {
                System.err.println("Thread was interrupted while sleeping");
            }
        }
    });
}

for(int i=0; i<5; i++){
    try {
        threads[i].join();
    } catch (InterruptedException e) {
        System.err.println("Thread was interrupted while waiting for thread " + i + " to finish");
    }
}

System.out.println("All done");
```

### Synchronization
Synchronization prevents some form of errors which happen if multiple threads try to access the same resource at the same time. As discussed above, the situation can lead to:
**Thread Interference:** two operations consist of multiple steps, and the sequences of steps overlap. As in the counter example above.  
**Memory Inconsistency:** errors occur when different threads have inconsistent views of what should be the same data.

**Synchronized Methods** whenever we add `synchronized` keyword to a method, the method can only be executed by one thread at a time. The counter program can be rewritten by using synchronized methods as:
```java
class SharedObject {
    private long counter = 0;

    public synchronized void incrementCounter() {
        this.counter++;
    }

    public synchronized long getCounter() {
        return this.counter;
    }
}
```

It should be noted that Constructors cannot be synchronized.

**Synchronized Blocks** synchronization is built around the concept of *locks*. Before a thread can execute a synchronized method, it must own the lock associated with it. In case of synchronized methods, that lock is `this` reference. This can be better illustrated by rewriting the above methods as:
```java
class SharedObject {
    private long counter = 0;

    public void incrementCounter() {
        synchronized(this){
            this.counter++;
        }
    }

    public long getCounter() {
        synchronized(this){
            return this.counter;
        }
    }
}
```

One possible problem with synchronized method is that out of all the synchronized methods of an object instance, only one method can be run at a time. In the above example two different threads therefore cannot execute two different synchronized methods at the same time. Therefore, in certain scenarios where we we want to modify two completely unrelated data in an object, we make use of synchronized blocks with multiple locks for more granular control over concurrent access:
```java
class SharedObject {
    private long c1 = 0;
    private long c2 = 0;
    private Object o1 = new Object();
    private Object o2 = new Object();

    public void incrementC1() {
        synchronized(o1){
            this.c1++;
        }
    }
    
    public void incrementC2() {
        synchronized(o2){
            this.c2++;
        }
    }
}
```

### Threads
There are two primary ways to create a thread. 1) Extend the `Thread` class and override the run method.

In [ ]:
public class Worker extends Thread{
    @Override
    public void run(){
        System.out.println("Worker thread running");
    }
}

Worker worker = new Worker();

2) The preferred way is to pass a `Runnable` instance to Thread constructor

In [ ]:
Runnable runnable = new Runnable{
    public void run(){
        System.out.println("Worker thread running");
    }
}

Thread worker = new Thread(runnable);

**Starting a thread:** use the `start` method of the Thread class. The `isAlive` method can tell if the thread is currently running or not. isAlive returns false if the thread has not been started or the thread has been terminated.  

**Stopping a thread:** is not as straight forward as starting it. The only way to terminate a thread is to arrange for its run method to complete. The thread class has a `stop` method, but it has been deprecated. It has been deprecated because stopping a thread releases all the locks it may have. If the thread was inside a critical section when stopped, it would lead to corrupted state. Other threads which access that critical section might be working with corrupted state.

Thread throwing an unchecked exception can also terminate the thread. Two approaches for stopping a thread: 
1) setting a flag

In [ ]:
public class Worker extends Thread{
    private volatile boolean done = false;
    
    @Override
    public void run(){
        while(!done){
            // Do something
        }
    }
    
    // Other methods
}

An external code can stop this thread by setting done as true. But the thread will only stop when the execution reaches the top of the loop (and not immediately).  
2) Using interrupts: we can call `interrupt` method of any thread. It has two effects. First, it sets internal interrupted flag of the thread to true. Second, it causes any blocking method like `sleep`, `join`, `wait` to stop blocking and throw `InterruptedException`.

In [ ]:
public class Worker extends Thread{
    
    @Override
    public void run(){
        while(!isInterrupted()){
            // Do something
        }
    }
    
    // Other methods
}

Worker worker = new Worker();
worker.start();
worker.interrupt();

**Pausing a thread:** the Thread class has `suspend` and `resume` methods which have been deprecated (due to possibility of deadlock). It is still possible for a thread to suspend its own execution for a specified amount of time using `sleep` method. The least amount of time a thread can sleep on most Java implementations is 20 to 50 ms.

In [6]:
public class DelayedRandomGenerator extends Thread {
    public void run() {
        Random random = new Random();
        while (!isInterrupted()) {
            System.out.print(random.nextInt() + "\t");
            try {
                sleep(200);
            } catch (InterruptedException e) {
                break;
            }
        }
    }
}

DelayedRandomGenerator generator = new DelayedRandomGenerator();
generator.start();

try {
    Thread.sleep(1000);
} catch (InterruptedException e) {
    System.exit(0);
}        

generator.interrupt();

1152488192	2140810343	1285862342	1650368227	-1184878083	1039227028	278517272	-1543200418	401414008	-531716027	

### Race Condition
When two or more threads access a given state variable, and try to change it, it can lead to a race condition.

In [ ]:
class SharedUnsafeObject {
    private long counter = 0;

    public void incrementCounter() {
        this.counter++;
    }

    public long getCounter() {
        return this.counter;
    }
}

SharedUnsafeObject obj = new SharedUnsafeObject();

// 5 threads trying to access counter variable of the
// same object
for (int i = 0; i < 5; i++) {
    new Thread(() -> {
        obj.incrementCounter();
    }).start();
}

try {
    Thread.sleep(1000);
    System.out.println(obj.getCounter()); // Output not guaranteed as 5
} catch (InterruptedException e) {
    e.printStackTrace();
}

Race condition in above case happens because the operation `this.counter++;` is not atomic. It is comprised of few substeps and different threads can interleave execution of these steps. 

Another example of non-atomic operation: 32 bit read/write (int, references) operations are atomic, however long, float (64 bits) etc are not stored/loaded atomically, rather it is 2 32 bits store operations. For example:

```
ThreadA : long sharedX = 2L;
ThreadB : long sharedY = -1L;

Possible ordering
------------------------------------------
Thread A            | Thread B
------------------------------------------
setHi 0000 0000     | 
                    | setHi ffff ffff
setLo 0000 0002     | 
                    | setLo ffff ffff
```

 Another example of non-atomic operation is creating new object

In [ ]:
Point p = new Point(5,4);

The above statement is not atomic. It is composed of the following steps:
1. Allocate memory for the object
2. Call construct to initialise values

Illustrated as:
```
local1 = calloc(sizeof(Point));
local1.<init>(5,4);
    Object.<init>();
    local1.x = 5;
    local1.y = 4;
instance = local1;
```

In [ ]:
/* Inadequate singleton implementation. May end up with
   multiple instances of Singleton
*/
public class Singleton{
    private static Singleton instance = null;
    
    public Singleton getInstance(){
        if(instance == null){
            instance = new Singleton();
        }
        
        return instance;
    }
}

One way to prevent race condition is to utilise *atomic operations*. The increment operation in previous example is a *read-modify-write* operation, whereas operation in singleton example is *check then act* operation. Both of these must be atomic. The read-modify-write operation problem can be solved using an *Atomic Type*.

In [ ]:
class SharedSafeObject {
    // Atomic varibles ensure memory visibility
    private AtomicLong counter = new AtomicLong(0);

    public void incrementCounter() {
        this.counter.incrementAndGet();
    }

    public long getCounter() {
        return this.counter.get();
    }
}

Another solution is to use `synchronized` keyword. A method marked as synchronized can be executed by a single thread at any given moment. Whenever a thread wants to execute a synchronized method, it first obtain the corresponding lock. If the lock is held by another thread, then the thread requesting lock must wait till the lock is released.  

Below is the synchronized version of the shared object example used earlier:

In [ ]:
class SharedObject {
    private long counter = 0;

    public synchronized void incrementCounter() {
        this.counter++;
    }

    public synchronized long getCounter() {
        return this.counter;
    }
}

The above example is equivalent to:

In [ ]:
// The lock object is 'this'
class SharedObject {
    private long counter = 0;

    public void incrementCounter() {
        synchronized(this){
            this.counter++;
        }
    }

    public long getCounter() {
        synchronized(this){
            return this.counter;
        }
    }
}

If an object has two independent state variables, it better to use to different lock objects such that two different threads can access two states concurrently.

In [ ]:
class SharedObject {
    private long c1 = 0;
    private long c2 = 0;
    private Object o1 = new Object();
    private Object o2 = new Object();

    public void incrementC1() {
        synchronized(o1){
            this.c1++;
        }
    }

    public void incrementC2() {
        synchronized(o2){
            this.c2++;
        }
    }
}

### Visibility
Synchronization also has another significant, and subtle, aspect: memory visibility. We want not only to prevent one thread from modifying the state of an object when another is using it, but also to ensure that when a thread modifies the state of an object, other threads can actually see the changes that were made.  

In general, there is no guarantee that the reading thread will see a value written by another thread on a timely basis, or even at all.

In [ ]:
public class NoVisibility {
    private static boolean ready;
    private static int number;
    
    private static class ReaderThread extends Thread {
        public void run() {
            while (!ready)
                // Yield means the current thread is willing to relinquish its current use of processor
                // but it'd like to be scheduled back soon as possible. 
                Thread.yield();
            System.out.println(number);
        }
    }
    
    public static void main(String[] args) {
        new ReaderThread().start();
        number = 42;
        ready = true;
    }
}

The problem with above code is that it is not necessary that the updated values of number or ready (made by main thread), is visible to the ReaderThread. The ReaderThread may print 42, or the loop may run indefinitely.  

One reason for reading stale values is the cache of the CPU cores. Each core of modern CPUs has its own cache. So if the reading and writing thread runs on different cores the reading thread sees a cached value and not the value written by the writing thread.

The ReaderThread may also output 0. This can be due to *instruction reordering*. JVM may reorder two instructions which are not related to each other. For example:
```
a = b + c
d = a + e

l = m + n
y = x + z
```
can be reordered to:
```
a = b + c

l = m + n
y = x + z

d = a + e
```

It is not sufficient to synchronize just the setter. The getter must also be synchronized, else we may get stale value.

In [ ]:
public class SynchronizedInteger {
    private int value;
    
    public synchronized int get() { 
        return value; 
    }
    
    public synchronized void set(int value) { 
        this.value = value; 
    }
}

Unlike primitives like int, float and double are not read from memory in one atomic step. Rather, 32 bits are read at a time. So it may be possible to read 32 bits of fresh value and 32 bits of stale value.

**Volatile variables:** are another solution to visibility problem. Any read or write on volatile variables are done to the main memory rather than CPU caches. The visibility effects of volatile variables extend beyond the value of the volatile variable itself. When thread A writes to a volatile variable and subsequently thread B reads that same variable, the values of all variables that were visible to A prior to writing to the volatile variable become visible to B after reading the volatile variable.

The above example of SynchronizedInteger can modified to:

In [ ]:
public class VolatileInteger {
    private volatile int value;
    
    public int get() { 
        return value; 
    }
    
    public void set(int value) { 
        this.value = value; 
    }
}

Locking using synchronized blocks can guarantee both visibility and atomicity; volatile variables can only guarantee visibility. So, the below code wouldn't work as expected. We would either need syncronization or atomic equivalent.

In [ ]:
// There is still a race condition in the below code
class SharedObject {
    private volatile long counter = 0;

    public void incrementCounter() {
        this.counter++;
    }

    public long getCounter() {
        return this.counter;
    }
}

### Thread Notification
The `wait` and `notify` semantics allows threads to communicate among each other. Wait (and its overloaded variants) forces a thread to wait and notify signals a waiting thread that it can continue execution. Both methods must be called from within a synchronized block. Synchronized block because of the inherent race condition:
1. Thread A tests a condition and discovers it must wait.
2. Thread B sets the condition and calls notify() to inform A to resume execution. Because A is not yet waiting, nothing happens.
3. Thread A waits, by calling wait().
4. Because of the prior notify() call, A waits indefinitely.

The thread that calls wait (the waiting thread) and the thread that calls notify (the notification thread) must compete for the same lock. Either thread must call wait or notify via the same object on which they enter their synchronized contexts because wait tightly integrates with the lock. Prior to waiting, a thread executing wait releases the lock, which allows the notification thread to enter its synchronized context to set the condition and notify the waiting thread. Notify on the other hand doesn't release any lock. Since any Java object can act as lock, the wait and notify methods are the Object class' methods.

In [ ]:
class Data {
    private String data;
    private Object lock = new Object();
    private boolean sent = true;

    public void send(String data) {
        synchronized (lock) {
            while (!sent) {
                try {
                    lock.wait();
                } catch (InterruptedException e) {
                    e.printStackTrace();
                }
            }

            sent = false;
            this.data = data;

            lock.notifyAll();
        }
    }

    public String receive(){
        synchronized (lock) {
            while (sent) {
                try {
                    lock.wait();
                } catch (InterruptedException e) {
                    e.printStackTrace();
                }
            }

            sent = true;
            lock.notifyAll();

            return this.data;
        }
    }
}

Wait/notify forms the basis of producer consumer pattern.

In [ ]:
class Common {
    private int number;
    private boolean writeable = true;

    public synchronized void setNumber(int number) {
        // If writeable is false, this means the consumer
        // has not consumed the data, wait
        while (!writeable) {
            try {
                wait();
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }

        writeable = false;
        this.number = number;
        System.out.println("Number " + this.number + " set");

        notify(); // If we miss this all consumers will continue
                  // waiting. Here we notify all waiting consumers that
                  // that new data is ready
    }

    public synchronized int getNumber() {
        // Producer has not produced new data, so better wait
        while (writeable) {
            try {
                wait();
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }

        writeable = true;
        notify(); // Notify producer that data has been consumed
                  // they can stop waiting (after this method
                  // has been executed)

        System.out.println("Number " + this.number + " read");
        return this.number;
    }
}

class Producer extends Thread {
    private Common common;

    public Producer(Common common) {
        this.common = common;
    }

    @Override
    public void run() {
        // Produce even numbers sequentially
        for (int i = 0; i < 100; i++) {
            if (i % 2 == 0) {
                common.setNumber(i);
            }
        }
    }
}

class Consumer extends Thread {
    private Common common;

    public Consumer(Common common) {
        this.common = common;
    }

    @Override
    public void run() {
        // Consume all generated numbers
        for (int i = 0; i < 50; i++) {
            common.getNumber();
        }
    }
}

What if a waiting thread is never notified. There is another overloaded version `void wait(long timeout)` which waits only for a certain duration. It is important to note that the wait method is called from inside a while loop and not an if block. Why?:
1. Thread A calls a method that acquires the synchronization lock.
2. Thread A examines a state flag and determines that the data is not in the desired state.
3. Thread A calls the wait() method, which frees the lock.
4. Thread B calls a method that acquires the same synchronization lock.
5. Thread C calls a method that blocks waiting for the lock.
6. Thread B sets the state flag and calls the notifyAll() method.
7. Thread B finishes its method and frees the lock.
8. Thread C acquires the lock and proceeds to process the data; it sees that the data is in the desired state, so it processes the data and resets the state flag.
9. Thread C exits without needing to wait.
10. Thread A receives the notification and wakes up.

This above sequence is not the one we expected. Therefore it is necessary to check the condition after waking from wait.

### ThreadLocal
ThreadLocal enables us to create variables that can only be read and written by the same thread. Consider the code below:

In [1]:
class BirthdayService{
    public String getFormattedBirthdate(int userId){
        Date birthDate = getBirthdate(userId);
        return new SimpleDateFormat("yyyy-mm-dd").format(birthDate);
    }
    
    // More methods
}

class Demo{
    public static void main(String[] args){
        BirthdayService service = new BirthdayService();
        
        Thread t1 = new Thread(()->{
            String birthDate = service.getBirthdate(50);
            System.out.println(birthDate);
        });
        
        Thread t2 = new Thread(()->{
            String birthDate = service.getBirthdate(150);
            System.out.println(birthDate);
        });
        
        t1.start();
        t2.start();
    }
}

We created two `SimpleDateFormat` objects, one per each thread call. What about the following modification?

In [ ]:
class Demo{
    public static void main(String[] args){
        BirthdayService service = new BirthdayService();
        
        for(int i=0; i<1000; i++){
            Thread t = new Thread(()->{
                String birthDate = service.getBirthdate(i);
                System.out.println(birthDate);
            });

            t.start();
        }
    }
}

Now we are creating 1000 SimpleDateFormat objects. We are also creating 1000 new threads. As we'll see later, 1000 new threads are not ideal, they are expensive to create and consume memory. We have 1000 tasks, we may want to use only 10 threads to execute these 1000 tasks. We can't use the same shared SimpleFormat object, but we can use the same SimpleFormat per thread. That is where ThreadLocal comes into play. To maintain 10 threads, we can make use of thread pools:

In [ ]:
public class ThreadSafeFormatter {
    // We are extending ThreadLocal because we have to initialize the
    // SimpleDateFormat in a specific way
    public static ThreadLocal<SimpleDateFormat> simpleDateFormat = new ThreadLocal<SimpleDateFormat>() {
        @Override
        protected SimpleDateFormat initialValue() {
            return new SimpleDateFormat("yyyy-mm-dd");
        };
    };
}

class BirthdayService{
    public String getFormattedBirthdate(int userId){
        Date birthDate = getBirthdate(userId);
        return new ThreadSafeFormatter.simpleDateFormat.get().format(birthDate);
    }
    
    // More methods
}

// We'll be using a FixedThreadPool
class Demo{
    public static void main(String[] args){
        BirthdayService service = new BirthdayService();
        
        ExecutorService service = Executors.newFixedThreadPool(10);
        // Each thread pool thread will have the its own
        // SimpleDateFormat object for all the tasks it gets
        for(int i=0; i<1000; i++){
            service.execute(()->{
                String birthDate = service.getBirthdate(i);
                System.out.println(birthDate);
            });
        }
        
        // Waiting for executor to finish
    }
}

If the `get` method is being called for the first time (by a thread), it internally calls the `initialValue` method. Subsequent calls to get by the same thread, returns the same object (with same initialized value).  

An important point to remember is that whenever the object has been completely used by a thread, we need to remove that object from ThreadLocal.

In [ ]:
ThreadSafeFormatter.simpleDateFormat.remove();

### Task Execution
Java provides a higher level concurrency framework called `Executor` framework. It decouples task submission from task execution. Executor is an interface with a simple `execute` method:

In [ ]:
public interface Executor {
    void execute(Runnable command);
}

The argument `command` contains the task that will be executed. Executors are based on *Producer-Consumer* pattern where producers submit task, and consumers execute task. Some simple implementation of Executor:

In [ ]:
class ThreadPerTask implements Executor{
    public void execute(Runnable command){
        new Thread(command).start();
    }
}

/* This is an example of unbounded thread creation. 
   The main problems with this approach are:
   - Thread creation and teardown is expensive. If a lot of tasks
     are submitted, this will consume significant computing resource
   - Active threads consume memory. A lot of idle threads
     would put pressure on garbage collector
   - JVMs often have limit on number of threads that can be created  
*/

Executor may not even make use of threads as illustrated:

In [ ]:
class SingleThread implements Executor{
    public void execute(Runnable command){
        command.run();
    }
}

Since Executors handle a bunch of tasks, it would be good if they provide lifecycle methods. They should be able to handle both graceful and forced shutdown. The `ExecutorService` interface extends `Executor` and provides a number of methods for lifecycle management and other convenience methods.

In [ ]:
public interface ExecutorService extends Executor {
    // Shutdown, stop accepting new tasks. However complete the tasks
    // already in queue. 
    void shutdown();
    
    // Forced shutdown, cancel any outstanding task
    List<Runnable> shutdownNow();

    // Return if the executor has been shutdown.
    boolean isShutdown();

    // All tasks have been completed
    boolean isTerminated();

    // Wait for ExecutorService to reach terminated status
    boolean awaitTermination(long timeout, TimeUnit unit)
        throws InterruptedException;
    
    // Other convenience methods
}

### Thread Pool
As we have seen, having unbounded thread creation approach isn't always helpful. It is beneficial to tie number of threads to the tasks that are being submitted. That is where *thread pools* come into picture. The idea is to maintain a pool of threads which can be reused, instead of creating new threads. Java provides `ThreadPoolExecutor` class which implements the ExecutorService interface.

In [ ]:
int  corePoolSize  =    5;
int  maxPoolSize   =   10;
long keepAliveTime = 5000;
BlockingQueue<Runnable> queue = getBlockingQueue();

ExecutorService exec = new ThreadPoolExecutor(
    corePoolSize,    
    maxPoolSize,
    keepAliveTime,
    queue
);

Core pool size is the target size, this number of threads will exist even if there is no task. New threads will not be created until the queue is full. Max pool size is the maximum number of threads that will exist at any given moment. A thread that has been idle for keepAliveTime will be terminated (if core pool size > max pool size).

In [1]:
int numberOfThreads = Runtime.getRuntime().availableProcessors() + 1;
System.out.print(numberOfThreads);

13

In case of IO intensive tasks, number of threads can be set higher. In general:
$$N_{threads} = N_{cpu}U_{cpu}(1 + \frac{W}{C})$$
Where
$$N_{cpu}\hspace{2em} \text{is number of CPUs}$$
$$U_{cpu}\hspace{2em} \text{is target CPU utilization, between 0 and 1}$$
$$\frac{W}{C}\hspace{2em} \text{is ratio of wait time to compute time}$$

The third parameter is a `BlockingQueue`. It represents a queue which is thread safe to put elements into, and take elements out of from. If a thread tries to take an element and there are none left in the queue, the thread can be blocked until there is an element to take. It is generally used when we have a scenario where one thread produces objects, which another thread consumes.

In [ ]:
public class BlockingQueueDemo {
    public static void main(String[] args) {
        BlockingQueue<Integer> queue = new LinkedBlockingQueue<>();

        Producer p = new Producer(queue);
        Consumer c = new Consumer(queue);

        new Thread(p).start();
        new Thread(c).start();
    }
}

class Producer implements Runnable {
    private BlockingQueue<Integer> queue;

    public Producer(BlockingQueue<Integer> queue) {
        this.queue = queue;
    }

    @Override
    public void run() {
        // Put 100 even numbers in queue
        for (int i = 0; i < 100; i++) {
            try {
                queue.put(i * 2);
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }
    }

}

class Consumer implements Runnable {
    private BlockingQueue<Integer> queue;

    public Consumer(BlockingQueue<Integer> queue) {
        this.queue = queue;
    }

    @Override
    public void run() {
        // Consume 100 integers
        for (int i = 0; i < 100; i++) {
            try {
                System.out.println(queue.take());
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }
    }

}

If we look at just the Executor interface, it does not enforce any queue requirement. ThreadPoolExecutor however, uses a blocking queue to store tasks. If the queue is unbounded, and a large number of tasks are recieved, it can lead to memory issues due to expanding queue. It is also possible that the ThreadPoolExecutor uses a large number of threads and the queue size remains low (or even not required, incoming tasks are directly assigned to a thread). Different BlockingQueue implementations are present which represent unbounded and bounded queues:
- `LinkedBlockingQueue` : unbounded queue
- `ArrayBlockingQueue` : bounded queue, need to specify its size
- `PriorityBlockingQueue` : unbounded, same priority as `PriorityQueue`
- `SynchronousQueue` : not a pure queue, contains a single item

What to do when we use a bounded queue and the queue gets saturated with tasks? For example, consider:

In [2]:
int coreCount = Runtime.getRuntime().availableProcessors();
ThreadPoolExecutor executor = new ThreadPoolExecutor(coreCount, coreCount * 2, 100, TimeUnit.MILLISECONDS,
        new ArrayBlockingQueue<Runnable>(50));

for (int i = 0; i < 500; i++) {
    final int taskNumber = i;
    executor.execute(() -> {
        try {
            Thread.sleep(200);
            System.out.println("Task number " + taskNumber + " completed by " + Thread.currentThread().getName());
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
    });
}

EvalException: Task REPL.$JShell$16$$Lambda$500/0x0000000800dab440@6816cfb8 rejected from java.util.concurrent.ThreadPoolExecutor@551e503d[Running, pool size = 24, active threads = 24, queued tasks = 50, completed tasks = 0]

We get `RejectedExecutionException` exception (in this case we will get min 50 system outs after exception). Another policy is to ignore new tasks. 

In [ ]:
int coreCount = Runtime.getRuntime().availableProcessors();
ThreadPoolExecutor executor = new ThreadPoolExecutor(coreCount, coreCount * 2, 100, TimeUnit.MILLISECONDS,
        new ArrayBlockingQueue<Runnable>(50));
executor.setRejectedExecutionHandler(new ThreadPoolExecutor.DiscardPolicy()); // or use the overloaded constructor

// This will loop 500 times, but not all tasks will be executed
for (int i = 0; i < 500; i++) {
    final int taskNumber = i;
    executor.execute(() -> {
        try {
            Thread.sleep(200);
            System.out.println("Task number " + taskNumber + " completed by " + Thread.currentThread().getName());
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
    });
}

// ThreadPoolExecutor.DiscardOldestPolicy discards the oldest tasks in queue, and
// queues in new tasks

Another saturation policy is `CallerRunsPolicy`. In this case, the calling thread (main thread in above example) will execute the task.

In [ ]:
int coreCount = Runtime.getRuntime().availableProcessors();
ThreadPoolExecutor executor = new ThreadPoolExecutor(coreCount, coreCount * 2, 100, TimeUnit.MILLISECONDS,
        new ArrayBlockingQueue<Runnable>(50));
executor.setRejectedExecutionHandler(new ThreadPoolExecutor.CallerRunsPolicy());

// This will loop 500 times, all tasks will be completed
// some by thread pool threads, other by main thread
for (int i = 0; i < 500; i++) {
    final int taskNumber = i;
    executor.execute(() -> {
        try {
            Thread.sleep(200);
            System.out.println("Task number " + taskNumber + " completed by " + Thread.currentThread().getName());
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
    });
}

Problem with above approach in the example code is that the task being executed by main thread will block new tasks from being queued.

Java provides some built-in thread pool implementations:

In [ ]:
// --- Fixed Thread Pool ---
ThreadPoolExecutor fTP = (ThreadPoolExecutor) Executors.newFixedThreadPool(10);
/* Equivalent to:
new ThreadPoolExecutor(nThreads, nThreads,
                       0L, TimeUnit.MILLISECONDS,
                       new LinkedBlockingQueue<Runnable>()); */

// --- Cached Thread Pool ---
ThreadPoolExecutor cTP = (ThreadPoolExecutor) Executors.newCachedThreadPool();
/* Equivalent to:
new ThreadPoolExecutor(0, Integer.MAX_VALUE,
                       60L, TimeUnit.SECONDS,
                       new SynchronousQueue<Runnable>()); */

// --- Single Thread Executor ---
ThreadPoolExecutor sTP = (ThreadPoolExecutor) Executors.newSingleThreadExecutor();
/* Equivalent to:
new ThreadPoolExecutor(1, 1,
                       0L, TimeUnit.MILLISECONDS,
                       new LinkedBlockingQueue<Runnable>())) */

### Callable and Future
The run method of a runnable doesn't return any value. A callable can be thought of as a runnable with return value. Consider the Callable which returns a value:

In [ ]:
class Task implements Callable<Integer> {
    @Override
    public Integer call() throws Exception {
        return new Random().nextInt();
    }
}

As we can see, callables can also throw checked exceptions, something runnables can't.  

ExecutorService has a `submit` method which accepts a Callable and returns a `Future`. A future is a placeholder for a value which will be returned by the callable sometimes in future. To get the value, we can call `get` method of the future, which is a blocking operation. It will wait till the future gets a value.  

If we have multiple tasks:

In [ ]:
ExecutorService eS = Executors.newFixedThreadPool(12);
List<Future<Integer>> futureList = new ArrayList<>();

for (int i = 0; i < 100; i++) {
    Future<Integer> randomFuture = eS.submit(new Task());
    futureList.add(randomFuture);
}

eS.shutdown();
eS.awaitTermination(1, TimeUnit.SECONDS);

// Retrieve values of all futures
for (int i = 0; i < 100; i++) {
    Future<Integer> randomFuture = futureList.get(i);
    System.out.println(randomFuture.get());
}

The problem with the above code is that getting value of future iterating through all futures is not optimal. There can be a possibility that for example, task 5 and 6 may have been completed (thus the futures have values), but since task 4 has not been completed, the call to get is blocking. We can avoid it by using an overloaded form of get which accepts a timeout.

### ForkJoinPool
`ForkJoinPool` is another type of executor service which utilises the fork join pattern. Fork/Join addresses the need for divide-and-conquer, or recursive task-processing in Java programs. Steps:
1. separate (fork) each large task into smaller tasks
2. process each task in a separate thread (separating those into even smaller tasks if necessary)
3. join the results

<img src="https://miro.medium.com/max/1400/0*jtObAMIwxoJRT9UG" width="600" height="auto">
<img src="https://miro.medium.com/max/1400/0*kGam9y6Y_A73Fd8m" width="600" height="auto">

Like a ThreadPoolExecutor, ForkJoinPool also maintains an internal thread pools. The default constructor creates ForkJoinPool with number of threads depending upon the cores in the machine. The other overloaded version of the constructor lets us specify number of threads.  

ForkJoinPool accepts `ForkJoinTask`. Concrete implementations:
- `RecursiveTask`
- `RecursiveAction`
- `CountedCompleter`
Once started, it will usually in turn start other subtasks.

### CompletableFuture
Is same as promise in JavaScript. To create a CompletableFuture,

In [8]:
CompletableFuture<Integer> completableFuture = CompletableFuture.supplyAsync(() -> {
    try{
        Thread.sleep(100);
    } catch(InterruptedException e) {
        System.err.println("Interrupted");
    }
    return 10;
});

completableFuture.thenAccept((val) -> {
    System.out.println("CompletableFuture resolved with value: " + val);
});

System.out.println("Hello from main");

Hello from main
